In [1]:
from llm import call_ai


print(call_ai(
    "You are a helpful assistant. Please answer the question: What is the capital of France?",
    'google'))

print(call_ai(
    "You are a helpful assistant. Please answer the question: What is the capital of France?",
    'openai'))
        

The capital of France is **Paris**.
The capital of France is Paris.


In [1]:
import templates
import loader  
from llm import call_ai
from datetime import datetime
from saver import save_user_imitation, save_evaluation_results
from logging_config import logger
#eval 
from eval import evaluate, evaluate_with_individual_scores

logger.info("Starting imitation generation...")
n= 3  # Number of stimuli to process, set to 3 for testing
#test_file_path = r'C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\534023.0.jsonl'
test_file_path = r'C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\4252893976.0.jsonl'
run_id =  str(datetime.now().strftime("%Y%m%d_%H%M%S"))
user_history = loader.get_formatted_user_historie(test_file_path)

formatted_User_history = templates.format_template('persona_template_simple', **{'historie':user_history})
persona=call_ai(formatted_User_history, 'google')
print(f"Persona for user 534023:\n{persona}")
#load stimulus

#stimulus , is_post, post_id=loader.load_stimulus(test_file_path)
all_stimuli=loader.load_stimulus(test_file_path)
for x in all_stimuli[:n]:  # Limiting to first 3 stimuli for testing
    stimulus, is_post, post_id = x
    print(f"Stimulus: {stimulus}, is_post: {is_post}, post_id: {post_id}")
    if is_post:
        print(f"Stimulus for user 534023 is post:\n{stimulus}")
        stimulus_post = templates.format_template('imitation_post_template_simple', **{'persona': persona,'tweet': stimulus})
    else:
        print(f"Stimulus for user 534023 is reply:\n{stimulus}")
        stimulus_reply = templates.format_template('imitation_replies_template_simple', **{'persona':persona,'tweet': stimulus})

    #call ai with the stimulus post
    if is_post:
        imitation = call_ai(stimulus_post, 'google')
        print(f"Imitation for user 534023 post:\n{imitation}")
    else:
        imitation = call_ai(stimulus_reply, 'google')
        print(f"Imitation for user 534023 reply:\n{imitation}")

    #save imitation
    save_user_imitation(file_path=test_file_path, stimulus= stimulus,persona=persona,imitation=imitation,run_id= run_id, tweet_id=post_id)



print("Starting evaluation...")
results=loader.load_predictions_orginales_formated(run_id=run_id, file_path=test_file_path)
print(f"Results for run_id {run_id}:")
result=evaluate_with_individual_scores(results) #average evaluation of all results
save_evaluation_results(file_path=test_file_path, evaluation_results=result, run_id=run_id)
print(f"Evaluation results saved for run_id {run_id}: {result}")


c:\Users\Christoph.Hau\Experimente\ha\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-06-16 15:27:38,896 - logging_config - INFO - Starting imitation generation...
2025-06-16 15:27:38,916 - logging_config - INFO - User history loaded from C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\4252893976.0.jsonl
Persona for user 534023:
Basierend auf den bereitgestellten Twitter-Posts des Nutzers "cindy_chojnicki" kann ein detailliertes Nutzerprofil erstellt werden:

-**DEMOGRAPHIE::** Weiblich (basierend auf dem Namen "cindy_chojnicki" und dem Kontext), wird in zwei Monaten 57 Jahre alt.
-**POLITISCHE_AUSRICHTUNG::** Stark pro-demokratisch und liberal. Unterstützt Präsident Biden und seine Regierung ("Bidenomics"). Ist eine Verfechterin des "Blauen Lagers" und der "Widerstandsbewegung" ("#BlueStormBoosts", "#wtpBLUE", "GM Resisters", "connectblue", "DemVoice1").
-**POLITISCHE_GEGNER::** Äußerst kritisch und ablehnend gegenüber Donald Trump, Ron DeSantis, Marjorie Taylor Greene und der MAGA-Bewegung. Bezeichnet Trump als "lying, crimping scumbag" und die MAGA-Extremis

INFO:logging_config:Added new evaluation for run_id: 20250616_152738


2025-06-16 15:30:17,147 - logging_config - INFO - Evaluation results successfully saved to C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\4252893976.0.jsonl


INFO:logging_config:Evaluation results successfully saved to C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\4252893976.0.jsonl


Evaluation results saved for run_id 20250616_152738: {'overall': {'rouge': {'rouge1': np.float64(0.45045045045045046), 'rouge2': np.float64(0.38382816826795735), 'rougeL': np.float64(0.45045045045045046), 'rougeLsum': np.float64(0.45045045045045046)}, 'bleu': {'bleu': 0.13177613267321317, 'precisions': [0.24661246612466126, 0.1557377049180328, 0.1046831955922865, 0.075], 'brevity_penalty': 1.0, 'length_ratio': 2.975806451612903, 'translation_length': 369, 'reference_length': 124}, 'total_samples': 3}, 'individual_scores': [{'index': 0, 'prediction': 'Hier ist der vervollständigte Tweet, der auf dem bereitgestellten Persona von "cindy_chojnicki" basiert:\n\n**Tweet:** Post: **Should Trump wiggle his way** into some type of **shady** deal, **he\'s still** going to **lose**. And as things **stand** at this moment, there is not a **chance** in hell that he can win a **general election**, even if members of his party are **delusional** enough to give him the **nomination**. That **POS**… **

In [3]:
import json
import os
from logging_config import logger

def load_results_for_reflection(run_id, file_path):
    """
    Load results for reflection based on run_id.
    
    :param run_id: Run ID to filter the results.
    :param file_path: Path to the JSONL file containing the user data.
    :return: Dictionary with formatted predictions, originals and eval results for the specified run_id.
    """
    if not os.path.exists(file_path):
        logger.error(f"File not found: {file_path}")
        return None
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            
        if len(lines) < 4:
            logger.error(f"File {file_path} has fewer than 4 lines")
            return None
        
        # Parse line 3 (index 2) for persona data
        try:
            persona_data = json.loads(lines[2].strip())
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding JSON from line 3 in {file_path}: {e}")
            return None
        
        # Parse line 4 (index 3) for evaluation data  
        try:
            eval_data = json.loads(lines[3].strip())
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding JSON from line 4 in {file_path}: {e}")
            return None
        
        # Get user_id (should be in both, but let's prioritize eval_data)
        user_id = eval_data.get('user_id') or persona_data.get('user_id')
        
        # Find the specific run in persona data
        runs = persona_data.get('runs', [])
        target_run = None
        for run in runs:
            if run.get('run_id') == run_id:
                target_run = run
                break
        
        if not target_run:
            logger.error(f"Run ID {run_id} not found in persona data")
            return None
        
        # Find the evaluation for this run
        evaluations = eval_data.get('evaluations', [])
        target_evaluation = None
        for evaluation in evaluations:
            if evaluation.get('run_id') == run_id:
                target_evaluation = evaluation
                break
        
        if not target_evaluation:
            logger.error(f"Evaluation for run ID {run_id} not found")
            return None
        
        return {
            'user_id': user_id,
            'run_id': run_id,
            'persona': target_run.get('persona', ''),
            'evaluation_results': target_evaluation.get('evaluation_results', {}),
            'timestamp': target_evaluation.get('timestamp', '')
        }
        
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        return None

# Test der Funktion
if __name__ == "__main__":
    # Beispiel mit den bereitgestellten Daten
    file_path = r"C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\4252893976.0.jsonl"  # Passe den Pfad an deine JSONL-Datei an
    run_id = "20250616_152738"
    
    data = load_results_for_reflection(run_id, file_path)
    
    if data:
        print("✅ Daten erfolgreich geladen!")
        print(f"User ID: {data['user_id']}")
        print(f"Run ID: {data['run_id']}")
        print(f"Persona (ersten 100 Zeichen): {data['persona'][:100]}...")
        print(f"Timestamp: {data['timestamp']}")
        
        # Zeige einige Metriken
        overall = data['evaluation_results'].get('overall', {})
        rouge = overall.get('rouge', {})
        bleu = overall.get('bleu', {})
        statistics = data['evaluation_results'].get('statistics', {})
        
        print(f"\n📊 Evaluation Metrics:")
        print(f"ROUGE-1: {rouge.get('rouge1', 'N/A'):.4f}" if rouge.get('rouge1') else "ROUGE-1: N/A")
        print(f"BLEU: {bleu.get('bleu', 'N/A'):.4f}" if bleu.get('bleu') else "BLEU: N/A")
        print(f"Mean Score: {statistics.get('mean_combined_score', 'N/A'):.4f}" if statistics.get('mean_combined_score') else "Mean Score: N/A")
        print(f"Total Samples: {overall.get('total_samples', 'N/A')}")
        
        # Test: Prüfe ob best_predictions und worst_predictions vorhanden sind
        eval_results = data['evaluation_results']
        best_preds = eval_results.get('best_predictions', [])
        worst_preds = eval_results.get('worst_predictions', [])
        individual_scores = eval_results.get('individual_scores', [])
        
        print(f"\n📈 Detailed Results:")
        print(f"Best predictions: {len(best_preds)} items")
        print(f"Worst predictions: {len(worst_preds)} items") 
        print(f"Individual scores: {len(individual_scores)} items")
        
        if best_preds:
            print(f"Best score: {best_preds[0].get('combined_score', 'N/A'):.4f}")
        if worst_preds:
            print(f"Worst score: {worst_preds[-1].get('combined_score', 'N/A'):.4f}")
            
    else:
        print("❌ Fehler beim Laden der Daten")

✅ Daten erfolgreich geladen!
User ID: 4252893976.0
Run ID: 20250616_152738
Persona (ersten 100 Zeichen): Basierend auf den bereitgestellten Twitter-Posts des Nutzers "cindy_chojnicki" kann ein detaillierte...
Timestamp: 2025-06-16T15:30:17.144783

📊 Evaluation Metrics:
ROUGE-1: 0.4505
BLEU: 0.1318
Mean Score: 0.3213
Total Samples: 3

📈 Detailed Results:
Best predictions: 3 items
Worst predictions: 3 items
Individual scores: 3 items
Best score: 0.4893
Worst score: 0.0000
